<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter

# <center> Очистка данных на Python </center>


<center> [Ссылка на код в JupiterNotebook в браузере со всеми графиками] (https://nbviewer.org/github/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/Analysis_of_HeadHunter_info.ipynb) </center>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go

# Библиотеки для отображения графиков plotly через сайт nbviewer.org
from plotly.offline import plot, iplot, init_notebook_mode
py.init_notebook_mode(connected=False)
import plotly.io as pio

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
hh_data = pd.read_csv('data/dst-3.0_16_1_hh_database.zip')

hh_data.shape

#### Ответ: Изначальный размер таблицы (44744, 12): 44744 - строки, 12 - столбцов-признаковю

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
display(hh_data.head())
display(hh_data.tail())

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
display(hh_data.info())

#### Ответ: В признаке "Опыт работы" наблюдается большое количество пустых значений и единичные пустые значения в признаках "Последнее/нынешнее место работы" и "Последняя/нынешняя должность"

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
display(hh_data.describe())

#### Ответ: Самая распространенная желаемая должность - Системный администратор.

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
# создадим функцию для определения уровня образования по первым словам
def education_level(arg):
    if arg.startswith('Высшее'):
        return 'высшее'
    elif  arg.startswith('Неоконченное высшее'):
        return 'неоконченное высшее'
    elif  arg.startswith('Среднее специальное'):
        return 'среднее специальное'
    elif  arg.startswith('Среднее'):
        return 'среднее'

# создадим новый признак и проверим его
hh_data['Образование'] = hh_data['Образование и ВУЗ'].apply(education_level)
display(hh_data['Образование'].unique())

hh_data[hh_data['Образование'] == 'среднее'].shape[0] # 559

hh_data = hh_data.drop(columns=['Образование и ВУЗ'], axis = 1)

#### Ответ: Число лиц со средним (школьным) образованием - 559.

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
# создадим функции преобразующую признак "Пол, возраст" в отдельные признаки
def gender(arg):
    info_list = arg.split()
    
    if 'Женщина' in info_list:
        return 'Ж'
    elif 'Мужчина' in info_list:
        return 'М'
    else:
        return 'Нет'
    
def ages(arg):
    ages = int(arg.split(',')[1].split()[0])
    
    return  ages


hh_data['Пол'] = hh_data["Пол, возраст"].apply(gender)
hh_data['Возраст'] = hh_data["Пол, возраст"].apply(ages)

hh_data = hh_data.drop(columns=["Пол, возраст"], axis=1)

# определим количество женскеих резюме
display(round((hh_data[hh_data['Пол'] == 'Ж'].shape[0]) / (hh_data.shape[0]) * 100, 2)) # 19.07% 

# определим средний возраст соискателей
display(round(hh_data['Возраст'].mean(), 1)) # 32.3 года 

#### Ответ: Число женских резюме составило - 19.07%. Средний возраст соискателей составил 32.2 года.

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
# создадим функцию преобразующую опыт работы в количество месяцев
def experience_months(arg):
    
    if arg is np.nan or arg == 'Не указано':
        return None
    
    age_list = ['год', 'года', 'лет']
    month_list = ['месяц', 'месяцев', 'месяца']
    
    result = 0
    
    check_list = str(arg).split()[:6]
    
    for i in range(len(check_list)): 
        if check_list[i] in age_list:
            result = result + int(check_list[i-1])*12
        elif check_list[i] in month_list: 
            result = result + int(check_list[i-1])
            
    
    return result
    
    
hh_data["Опыт работы (месяц)"] = hh_data["Опыт работы"].apply(experience_months)


hh_data = hh_data.drop(columns=["Опыт работы"], axis=1)
# определим медианный опыт работы в месяцах
print(hh_data["Опыт работы (месяц)"].median()) # 100 


#### Ответ: Медианный опыт работы в месяцах составил 100.0

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
# создадим функцию для указания города
def city_name(arg):
    million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск',
                  'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    
    city = arg.split()[0].strip()
    
    if city == 'Москва' or city == 'Санкт-Петербург':
        return city
    elif city in million_cities:
        return "город-миллионник"
    else:
        return 'другие'
    
# создадим функции о готовности к переезду
def migration_ready(arg):
    info_list = arg.split(',')
    
    #определим "нужный" кусок информации и проверим
    for info in info_list:
        if 'перее' in info:
            return ("не" not in info)
        
# создадим функции о готовности к командировкам
def moving_ready(arg):
    if 'команд'not in arg:
        return False
    
    info_list = arg.split(',')
    
    #определим "нужный" кусок информации и проверим
    for info in info_list:
        if 'команд' in info:
            return ("не" not in info)
        
hh_data['Город'] = hh_data["Город, переезд, командировки"].apply(city_name)
hh_data['Готовность к переезду'] = hh_data["Город, переезд, командировки"].apply(migration_ready)
hh_data['Готовность к командировкам'] = hh_data["Город, переезд, командировки"].apply(moving_ready)

# определим сколько % соискателей проживают в Санкт_Петербурге
display(round((hh_data[hh_data['Город'] == 'Санкт-Петербург'].shape[0]) / hh_data.shape[0] * 100, 0)) # 11

# определим сколько % соискателей готовы и к переезду, и к командировкам
display(round(((hh_data[(hh_data['Готовность к переезду'] == True) & 
                (hh_data['Готовность к командировкам'] == True)].shape[0]) / hh_data.shape[0]) * 100, 0)) # 32


hh_data = hh_data.drop(columns=["Город, переезд, командировки"], axis=1)
                

#### Ответ: 11% соискателенй проживает  в Санкт-Петербурге. Из всех соискателей 32 % готовы как к переезду, так и к командировкам.

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
# создадим трансформирующую функцию
def one_hot_transform(arg, col):
        return col in arg

# трансформируем столбцы "Занятость" и "График"
hh_data['полная занятость'] = hh_data['Занятость'].apply(one_hot_transform, col='полная занятость')
hh_data['частичная занятость'] = hh_data['Занятость'].apply(one_hot_transform, col='частичная занятость')
hh_data['проектная работа'] = hh_data['Занятость'].apply(one_hot_transform, col='проектная работа')
hh_data['волонтерство'] = hh_data['Занятость'].apply(one_hot_transform, col='волонтерство')
hh_data['стажировка'] = hh_data['Занятость'].apply(one_hot_transform, col='стажировка')

hh_data['полный день'] = hh_data['График'].apply(one_hot_transform, col='полный день')
hh_data['сменный график'] = hh_data['График'].apply(one_hot_transform, col='сменный график')
hh_data['гибкий график'] = hh_data['График'].apply(one_hot_transform, col='гибкий график')
hh_data['удаленная работа'] = hh_data['График'].apply(one_hot_transform, col='удаленная работа')
hh_data['вахтовый метод'] = hh_data['График'].apply(one_hot_transform, col='вахтовый метод')

# определим сколько человек ищут проектную работу и волонтёрство
print(hh_data[(hh_data['проектная работа'] == True) & (hh_data['волонтерство'] == True)].shape[0]) # 436

# определим сколько человек хотят работать вахтовым методом и с гибким графиком
print(hh_data[(hh_data['вахтовый метод'] == True) & (hh_data['гибкий график'] == True)].shape[0]) # 2311

hh_data = hh_data.drop(columns=['Занятость'], axis=1)
hh_data = hh_data.drop(columns=['График'], axis=1)

#### Ответ: Проектную работу и волонтёрство одновременно рассматривают 436 человек. Работать вахтовым методом с гибким графиком хотят 2311 человек.

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# откроeм дата фрейм с данныме о курсе валют
ex_rates = pd.read_csv('data/ExchangeRates.csv', sep=',')
ex_rates = ex_rates[['currency', 'date', 'close', 'proportion']]

# приведем даты в формат datetime
ex_rates['date'] = pd.to_datetime(ex_rates['date'], dayfirst=True)
hh_data['Обновление резюме'] = pd.to_datetime(hh_data['Обновление резюме'], dayfirst=True).dt.normalize()

# создадим функцию для перевода наименования валюты к стандарту ISO
def money_name(arg):
    
    salary = arg.split()[-1].replace('.', '')
    
    if salary == 'грн':
        salary = 'UAH'
    elif salary == 'белруб':
        salary = 'BYN'
    elif salary == 'сум':
        salary = 'UZS'
    elif salary == 'руб':
        salary = 'RUB'
    else:
        pass 
    
    return salary
# создадим функцию для численного значения зарплаты
def get_sum(arg):
    return int(arg.split()[0])

hh_data['ЗП_ISO'] = hh_data['ЗП'].apply(money_name)

hh_df_sal = hh_data.merge(ex_rates, how = 'left', left_on=['ЗП_ISO', 'Обновление резюме'], 
                          right_on=['currency', 'date'])

hh_df_sal.loc[hh_df_sal['ЗП_ISO'] == 'RUB', 'close'] = 1
hh_df_sal.loc[hh_df_sal['ЗП_ISO'] == 'RUB', 'proportion'] = 1



hh_df_sal["ЗП (руб)"] = hh_df_sal['ЗП'].apply(get_sum) * hh_df_sal['close'] / hh_df_sal['proportion']

# уберем лишние и вспомогательные столбцы
hh_data = hh_df_sal.drop(columns=['ЗП', 'ЗП_ISO', 'date', 'close', 'currency', 'proportion'], axis=1)

# определим медианное значение зарплаты в тысячах рублей
display(hh_data["ЗП (руб)"].median()) # 59

#### Ответ: Медианное значение зарплаты около 59 тысяч рублей.

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
box_age = px.box(data_frame=hh_data,
             x='Возраст',
             height=400,
             width=800,
             title='Распределение возраста соискателей')

# для изучения распределения посторим гистограмму
hist_age = px.histogram(data_frame=hh_data,
                    x='Возраст',
                    height=400,
                    width=800,
                    title='Распределение возраста соискателей')

box_age.show("notebook")
hist_age.show("notebook")

# изображение для GitHub
box_age.write_image('pictures/диаграмма1.png',engine="kaleido")
hist_age.write_image('pictures/диаграмма2.png',engine="kaleido")

box_age.show("notebook")
hist_age.show("notebook")

<center> <img src = https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B01.png alt="drawing" style="width:400px;">

<center> <img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B02.png alt="drawing" style="width:400px;">

#### Выводы: Распределение признака положительно скошенное (небольшой сдвиг влево). Возраст соискателей в пределах 14 - 77 лет (предельные значения признака). Большая часть соискателей в возрасте 27-36 лет (межквартильный размах 27-36 лет, медиана приходится на возраст 31 год, мода на 30 лет). Возраст старше 49 лет встречается в единичных случаях. Возраст 100 лет может быть отнесен к аномалиям (выбросам).

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
box_expirience = px.box(data_frame=hh_data,
             x='Опыт работы (месяц)',
             height=400,
             width=800,
             title='Распределение признака - опыт работы (в месяцах)')

# для изучения распределения построим гистограмму
hist_expirience = px.histogram(data_frame=hh_data,
                    x='Опыт работы (месяц)',
                    nbins= 200,
                    height=400,
                    width=800,
                    title='Распределение признака - опыт работы (в месяцах)')

box_expirience.show("notebook")
hist_expirience.show("notebook")

# изображение для GitHub
box_expirience.write_image('pictures/диаграмма3.png',engine="kaleido")
hist_expirience.write_image('pictures/диаграмма4.png',engine="kaleido")

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B03.png alt="drawing" style="width:400px;">

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B04.png alt="drawing" style="width:400px;">

#### Выводы: распределение признака похож на логнормальное, мода приходится на опыт работы 80-89 месяца (2720 соискателей), в то время как медиана на уровне 100 месяцев. Большая часть соискателей имеет опыт 57-154 месяца. Опыт работы более 300 месяцев встречается редко, а опыт более 400 месяцев нужно изучить на предмет отнесения к аномалиям (выбросам). Максимальный опыт работы - явный выброс - около 100 лет (1188 месяцев).

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака желаемой зарплаты? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
box_salary = px.box(data_frame=hh_data,
             x='ЗП (руб)',
             height=400,
             width=800,
             title='Уровень желаемой зарплаты (в рублях)')

# для изучения распределения построим гистограмму
hist_salary = px.histogram(data_frame=hh_data,
                    x='ЗП (руб)',
                    height=400,
                    width=800,
                    title='Уровень желаемой зарплаты (в рублях)')

box_salary.show("notebook")
hist_salary.show("notebook")

# изображение для GitHub
box_salary.write_image('pictures/диаграмма5.png',engine="kaleido")
hist_salary.write_image('pictures/диаграмма6.png',engine="kaleido")

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B05.png alt="drawing" style="width:400px;">

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B06.png alt="drawing" style="width:400px;">

In [ ]:
# рассмотрим зарплаты в пределах от 10 тысяч до 1 миллиона
salar_df = hh_data[(hh_data['ЗП (руб)'] < 1000000) & (hh_data['ЗП (руб)'] > 10000)]
box_salary2 = px.box(data_frame=salar_df,
             x='ЗП (руб)',
             height=400,
             width=800,
             title='Уровень желаемой зарплаты до 1 млн (в рублях)')

# для изучения распределения построим гистограмму
hist_salary2 = px.histogram(data_frame=salar_df,
                    x='ЗП (руб)',
                    nbins=100,
                    height=400,
                    width=800,
                    title='Уровень желаемой зарплаты до 1 млн (в рублях)')

box_salary2.show("notebook")
hist_salary2.show("notebook")

# изображение для GitHub
box_salary2.write_image('pictures/диаграмма7.png',engine="kaleido")
hist_salary2.write_image('pictures/диаграмма8.png',engine="kaleido")


<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B07.png alt="drawing" style="width:400px;">

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B08.png alt="drawing" style="width:400px;">

#### Выводы: Следует отметить, что большая часть соискателей рассчитывает на зарплату 40-100 тысяч (межквартильный размах 40-95 тыс. рублей). Разброс (предельные значения признака) от 1 рубля до 24 млн рублей. В силу полученных результатов, данные по уровню зарплаты  более 1 млн рублей (5 соискателей) можно считать аномалиями, вероятно, выбросы.

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
salar_less_df = hh_data[(hh_data['ЗП (руб)'] < 1000000)]

box_education_sal = px.box(data_frame=salar_less_df,
                    y='ЗП (руб)',
                    x='Образование',
                    color='Образование',
                    width=800,
                    height=600,
                    title='Зависимость уровня желаемой зарплаты (в рублях) от уровня образования')
                    
box_education_sal.show("notebook")

# изображение для GitHub
box_education_sal.write_image('pictures/диаграмма9.png',engine="kaleido")

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B09.png alt="drawing" style="width:400px;">


#### Вывод: отмечается прямая зависимость желаемой заработной платы от уровня образования соискателя. Наибольшую заработную плату желают получить соискатели, имеющие высшее образование (медиана 60 тыс. рублей), чуть менее соискатели с незаконченным высшим образованием (медиана 50 тыс. рублей), еще меньше уровень притязаний у лиц, имеющих среднее образование, и наименьшие заработные платы ожидают соскатели со средним специальным образованием (хотя уровень медианы в обеих категориях  40 тыс. рублей). Следовательно, уровень образования может использоваться для прогнозирования уровня заработной платы.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
salar_less_df = hh_data[(hh_data['ЗП (руб)'] < 1000000)]

box_city_sal = px.box(data_frame=salar_less_df,
                    y='ЗП (руб)',
                    x='Город',
                    color='Город',
                    width=800,
                    height=600,
                    title='Зависимость уровня желаемой зарплаты (в рублях) от города предполагаемой работы')
                    
box_city_sal.show("notebook")

# изображение для GitHub
box_city_sal.write_image('pictures/диаграмма10.png',engine="kaleido")

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B010.png alt="drawing" style="width:400px;">

#### Вывод: имеется явная зависимость желаемой заработной платы от города предполагаемой работы. Наибольшую заработную плату желают получить соискатели, пребывающие в Москве (медиана - 85 тыс. рублей, межквартильный размах 60-150 тыс. рублей), чуть менее соискатели в Санкт-Петербурге (медина - 60 тыс. рублей, межквартильный размах 45-85 тыс. рублей), различия в других города (между городами-миллионниками и меньшими городами) незначительны (медиана - 40 тыс. рублей, межквартильный размах 30-60 тыс. рублей). 
Следует отметить, что в Москве и городах с населением менее 1 млн среди уровней желаемой зарплаты встречаются пожелания более 600 тыс. рублей за свою работу. И если для Москвы с учетом преобладания более высококвалифицированных кадров с изначально высоким уровнем притязаний данные факты могут соответствовать действительно высокому профессиональному уровню, то для городов с числом жителей менее 1 млн (максимальное значение около 924 тыс. руб) эти данные скорее аномальны, и должны изучатся на предмет выявления выбросов. Следовательно, уровень образования может использоваться для прогнозирования более высокого уровня заработной платы в Москве и Санкт-Петербурге, чем в других городах РФ.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
# создадим новый датафрейм для трансформации данных
hh_df = hh_data[['Готовность к переезду', 'Готовность к командировкам', 'ЗП (руб)']]
hh_df.loc[hh_df['Готовность к переезду'] == True, 'Готовность к переезду' ] = "готов(а)"
hh_df.loc[hh_df['Готовность к переезду'] == False, 'Готовность к переезду' ] = "не готов(а)"

hh_df.loc[hh_df['Готовность к командировкам'] == True, 'Готовность к командировкам' ] = "готов(а)"
hh_df.loc[hh_df['Готовность к командировкам'] == False, 'Готовность к командировкам' ] = "не готов(а)"
display(hh_df.head())

# по отдельности сгруппируем данные по зарплате
move_df = hh_df.groupby('Готовность к переезду', as_index=False)[['ЗП (руб)']].median()
display(move_df)

com_df = hh_df.groupby('Готовность к командировкам', as_index=False)['ЗП (руб)'].median()
display(com_df)


import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Histogram(histfunc='sum',x=move_df['Готовность к переезду'], y=move_df['ЗП (руб)'], 
                           name='Переезд'))
fig.add_trace(go.Histogram(histfunc='sum', x=com_df['Готовность к командировкам'], y=com_df['ЗП (руб)'], 
                           name='Командировки'))

fig.update_layout(height=500, width = 800, 
                  title='Зависимость желаемого уровня зарплаты от готовности к переезду и командировкам')

fig.show("notebook")

# изображение для GitHub
fig.write_image('pictures/диаграмма11.png',engine="kaleido")

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B011.png alt="drawing" style="width:400px;">

#### Вывод: Готовность как к переезду, так и к командировкам способствует более высоким требованиям к желаемой заработной плате (медиана 65 тыс. рублей). Больше всего на снижение уровня заработной платы влияет неготовность ездить в командировки.

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
pivot_salary = hh_data.pivot_table(values='ЗП (руб)', 
                                   index='Возраст', 
                                   columns='Образование', 
                                   aggfunc='median')

display(pivot_salary)

plt.figure(figsize=(15,10))

heatmap_pivot_salary = sns.heatmap(data=pivot_salary,
                                   cmap='YlGnBu')
heatmap_pivot_salary.set_title('Уровень ожидаемой зарплаты (в рублях) в зависимости от уровня образования и возраста',
                               fontsize=16);


#### Выводы: 

* Как уже отмечалось ранее больший уровень желаемой заработной платы наблюдается среди лиц с высшим образованием. 

* Среди лиц с высшим  образованием в большинстве своем наблюдается возрастание (самое быстрое из всех групп) уровня желаемой зарплаты к 40 годам (38-45 лет) и последующим снижением за исключением возраста младше 18 лет. Следует отметить, что возраст до 18 - 21 для оконченного высшего образование является нетипичным, и такие резюме нуждаются в отдельной оценке. В возрасте старше 60 лет четкой зависимости не наблюдаеся. Резкий всплеск приходится на 69 лет. Есть пропуски данных в возрасте 15 лет.

* Среди лиц с неоконченным высшим образованием в большинстве своем наблюдается возрастание уровня желаемой зарплаты к 43 годам за исключением возраста младше 18 лет (также нетипичный возраст) и последующим снижением. Резкий всплеск приходится на 58 лет.

* Для лиц со средним образованием зависимость между возрастом и желаемой заработной платой не выражена. Резкий всплеск приходится на 56 лет. Есть пропуски данных в возрасте 16 лет и 51 год.

* Для лиц со средним специальным образованием зависимость выражена не так четко, с незначительным увеличением заработной платы к среднему возрасту (до 60 лет) и последующим снижением (за исключением пары "всплесков" в 67 и 100 лет). Общий уровень заработной платы самый низкий из всех групп (преимущественно до 60 тыс. руб). Есть пропуски данных. Есть множественные пропуски данных в возрасте старше 64 лет. И отдельный случай для соискателя в возрасте 100 лет (явный выброс).

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
scatter_data = hh_data[['Опыт работы (месяц)', 'Возраст']].copy()

# функция для переводы месяцев в полные года
def month_to_years(arg):
    return arg / 12

scatter_data['Опыт работы (лет)'] = hh_data['Опыт работы (месяц)'].apply(month_to_years)
display(scatter_data.head())

fig = go.Figure()

fig.add_trace(go.Scatter(x=scatter_data['Возраст'], 
                         y=scatter_data['Опыт работы (лет)'], 
                         mode='markers', 
                         name='Опыт/Возраст'))

fig.add_trace(go.Scatter(x=[0, 100], 
                         y=[0, 100],
                         name='Пограничное значение'))


fig.update_layout(height=500, 
                  width = 800,
                  legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title='Соотношения опыта работы и возраста соискателей')

fig.show("notebook")

# изображение для GitHub
fig.write_image('pictures/диаграмма12.png',engine="kaleido")


<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B012.png alt="drawing" style="width:400px;">

#### Вывод: Имеется прямая зависимоть между возрастом и опытом работы. В то же время на графике имеются данные по 7 соискателям (две сливаются), чей опыт работы соответствует или превосходит их возраст, следовательно эти данные относятся к аномалиям.

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
# оценим готовность к командировкам и к переезду в зависимости от возраста и пола
hh_df_add = hh_data.copy()
hh_df_add.loc[hh_df_add['Готовность к переезду'] == True, 'Готовность к переезду' ] = "готов(а)"
hh_df_add.loc[hh_df_add['Готовность к переезду'] == False, 'Готовность к переезду' ] = "не готов(а)"

hh_df_add.loc[hh_df_add['Готовность к командировкам'] == True, 'Готовность к командировкам' ] = "готов(а)"
hh_df_add.loc[hh_df_add['Готовность к командировкам'] == False, 'Готовность к командировкам' ] = "не готов(а)"

# демонстрация зависимости готовности к переезду от возраста и пола
hist_age_ready = px.histogram(data_frame=hh_df_add,
                              x='Возраст',
                              color='Готовность к переезду',
                              nbins=20,
                              width=800,
                              height=400,
                              title='Готовность к переезду в зависимости от возраста')

hist_age_ready.update_layout(barmode = 'group')

hist_age_ready.show("notebook")

# изображение для GitHub
hist_age_ready.write_image('pictures/диаграмма13.png',engine="kaleido")

# сводная таблица по склонности к переезду в зависимости от возраста и пола
pivot_gendere = hh_data.pivot_table(values='Готовность к переезду', index='Возраст', 
                                   columns='Пол', aggfunc='mean')

display(pivot_gendere)

plt.figure(figsize=(15,10))
heatmap_pivot_gender = sns.heatmap(data=pivot_gendere,
                                   cmap='YlGnBu')
heatmap_pivot_gender.set_title('Готовность к прееезду в зависимости от пола и возраста',
                               fontsize=16)

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B013.png>

In [ ]:
# посмотрим соотношение готовности к переезду в различных гендерных группах
treemap_gender_ready = px.treemap(data_frame=hh_df_add,
                                path=['Пол', 'Готовность к переезду'],
                                height=500,
                                width=800,
                                title ='Степень готовности к переезду среди соискателей по гендерному признаку'
)
treemap_gender_ready.show("notebook")

# изображение для GitHub
treemap_gender_ready.write_image('pictures/диаграмма14.png',engine="kaleido")

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B014.png alt="drawing" style="width:400px;">

In [ ]:
# демонстрация зависимости готовности к командировкам от возраста и пола
hist_age_activ = px.histogram(data_frame=hh_df_add,
                              x='Возраст',
                              color='Готовность к командировкам',
                              nbins=20,
                              width=800,
                              height=400,
                              title='Готовность к командировкам в зависимости от возраста')

hist_age_activ.update_layout(barmode = 'group')

hist_age_activ.show("notebook")

# изображение для GitHub
hist_age_activ.write_image('pictures/диаграмма15.png',engine="kaleido")


# сводная таблица по склонности к командировкам в зависимости от возраста и пола
pivot_gender_activ = hh_data.pivot_table(values='Готовность к командировкам', index='Возраст', 
                                   columns='Пол', aggfunc='mean')

display(pivot_gender_activ)

plt.figure(figsize=(15,10))
heatmap_pivot_gender2 = sns.heatmap(data=pivot_gender_activ,
                                   cmap='YlGnBu')
heatmap_pivot_gender2.set_title('Готовность к командировкам в зависимости от пола и возраста',
                               fontsize=16)



<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B015.png alt="drawing" style="width:400px;">

In [ ]:
# посмотрим соотношение готовности к командировкам в различных гендерных группах
treemap_gender_ready2 = px.treemap(data_frame=hh_df_add,
                                path=['Пол', 'Готовность к командировкам'],
                                height=500,
                                width=800,
                                title ='Степень готовности к командировкам среди соискателей по гендерному признаку'
)
treemap_gender_ready2.show("notebook")

# изображение для GitHub
treemap_gender_ready2.write_image('pictures/диаграмма16.png',engine="kaleido")

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B016.png alt="drawing" style="width:400px;">

#### Выводы: Готовность к переезду и командировкам зависит от пола и возраста соискателя. И мужчины, и женщины одинаково более готовы к командировкам, чем к переезду.

* Готовность к переезду у мужчин выше, чем у женщин, и несколько повышается в возрасте 20-48 лет. 
* Среди женщин отмечается незначительный подьем готовности к переезду в 23-34 года и резкий 18, 60 и 70 лет ("нет семьи" или "дети выросли").

* Мужчины демонстрируют высокую готовность к командировкам независимо от возраста (наибольшую до 60 лет). Смущают очень высокие показатели готовности  в возрасте около 100 лет - Выбросы?
* Среди женщин степень готовности к командировкам повышается (менее значимо, чем у мужчин) в возрасте 21-55 года. Отмечается неожиданные всплески готовности к командировкам в возрасте 64, 69-71 и 73 года. Учитывая возраст соискателей, сложно предположить, с чем это связано (возможно, выход на пенсию, смерть супруга итд) 

In [ ]:
# интересно посмотреть уровень зарплаты в зависимости от пола
gender_salary = hh_df_add.groupby(by ='Пол')['ЗП (руб)'].mean()

bar_gender_salary = px.bar(data_frame=gender_salary,
                 x=gender_salary.values,
                 y=gender_salary.index,
                 color=gender_salary.index,
                 height=500,
                 width=800,
                 title='Зависимость уровня желаемой зарплаты от возраста'
)
bar_gender_salary.show("notebook")

# изображение для GitHub
treemap_gender_ready2.write_image('pictures/диаграмма17.png',engine="kaleido")

<img src=https://github.com/EkaterinaArsa/DS_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20HeadHunter%20/pictures/%D0%B4%D0%B8%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B017.png alt="drawing" style="width:400px;">

#### Вывод: Средний уровень желаемой зарплаты у мужчин (80 тыс. руб) выше, чем у женщин (60 тыс. руб), что ожидаемо всвязи с их большей готовностью к переезду и командировкам.
Было бы интересно посмотреть, как сильно меняется зарплата от готовности к переезду и командировкам в зависимости от пола.

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
# создадим резервную копию
hh_data_zero = hh_data.copy()

duplicateRows = hh_data[hh_data.duplicated ()]
print(len(duplicateRows.index)) # 161 полный дупликат

# удалим полные дупликаты
hh_data = hh_data.drop_duplicates()
# и проверим результат
duplicateRows = hh_data[hh_data.duplicated ()]
print(len(duplicateRows.index)) # 0 полных дупликата

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
hh_data.isnull().sum()
# 1 пропуск в признаке 'Последнее/нынешнее место работы', 
# 2 пропуска в признаке 'Последняя/нынешняя должность' 
# 168 пропусков в признаке 'Опыт работы (месяц)'.

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
# заполним пропуски в признаке 'Опыт работы (месяц)'
hh_data['Опыт работы (месяц)'] = hh_data['Опыт работы (месяц)'].fillna(hh_data['Опыт работы (месяц)'].median())

# удалим строки с пустыми ячейками и проверим результат
hh_data = hh_data.dropna()
display(hh_data.info())

display(round(hh_data['Опыт работы (месяц)'].mean(), 0)) 
# среднее значение в столбце "Опыт работы (месяц)" 114

In [ ]:
display(hh_data.head())

# создадим резервную копию
hh_data_non_null = hh_data.copy()

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
hh_data_outliers = hh_data[(hh_data['ЗП (руб)'] < 1000) | (hh_data['ЗП (руб)'] > 1000000)]

hh_data_cleaned = hh_data[(hh_data['ЗП (руб)'] > 1000) & (hh_data['ЗП (руб)'] < 1000000)]

display(hh_data_outliers.info()) # 89


5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
hh_data_cleaned = hh_data_cleaned[hh_data_cleaned['Опыт работы (месяц)']/12 < hh_data_cleaned['Возраст']]
hh_data_outliers = hh_data_cleaned[hh_data_cleaned['Опыт работы (месяц)']/12 >= hh_data_cleaned['Возраст']]

display(hh_data_outliers.shape[0]) # 7 резюме, где возраст меньше опыта работы


6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
# создадим функцию определяющую выбросы методом трех сигм (вправо + 4 сигмы) 
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 4 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_z_score(hh_data_cleaned, 'Возраст', log_scale=True)
display(outliers['Возраст'])

# построим график для логарифмического распределения 
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

log_hh_data_cleaned= np.log(hh_data_cleaned['Возраст']) # 15 и 100 лет

histplot = sns.histplot(log_hh_data_cleaned, bins=30, ax=ax)
# добавим линии, отображающие среднее и границы интервала метода трех сигм
histplot.axvline(log_hh_data_cleaned.mean(), color='k', lw=2)
histplot.axvline(log_hh_data_cleaned.mean()+ 3 * log_hh_data_cleaned.std(), color='k', ls='--', lw=2)
histplot.axvline(log_hh_data_cleaned.mean()- 3 * log_hh_data_cleaned.std(), color='k', ls='--', lw=2)

histplot.set_title('Распределение признака возраста соискателей в логарифмическом масштабе');

#### Вывод: распределение признака "Возраст" в логарифмическом масштабировании положительно скошенное (сдвиг влево). В выбросы попало три резюме с возрастом соискателей 15 и 100 лет.